# 1. 필요 라이브러리 불러오기

In [1]:
import glob
import tensorflow as tf
import os, re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras

# 2. 데이터 가져오기

In [2]:
txt_file_path = '/content/drive/MyDrive/aiffel/Exp6/data/EXP6/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Let's stay together I, I'm I'm so in love with you", 'Whatever you want to do', 'Is all right with me']


# 3. 데이터 정제

위에서 확인 결과 공백 또는 문장끝이 : 인 문장은 없지만 혹시 모르니 처리하주겠다.

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Let's stay together I, I'm I'm so in love with you
Whatever you want to do
Is all right with me
Cause you make me feel so brand new
And I want to spend my life with you Let me say that since, baby, since we've been together
Loving you forever
Is what I need
Let me, be the one you come running to
I'll never be untrue Oh baby
Let's, let's stay together (gether)


#### 변경할 내용
- 대문자 -> 소문자
- 특수문자 [?.!,¿] 제외하고 제거
- 특수문자 [']도 i'ii / let's 등 때문에 제외
- 문장 앞부분과 끝부분에 <start>, <end> 추가

In [4]:
# sub(패턴,교체할문자열,문자열,최대교체수) : 패턴에 맞으면 그 문자를 교체할문자열로 교체는 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2. 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 3. 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿']+", " ", sentence) # 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다 + let's / i'll/ i'm과 같은 문자열을 위해 ' 도 추가
    sentence = sentence.strip() # 5. 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

In [5]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

["<start> let's stay together i , i'm i'm so in love with you <end>",
 '<start> whatever you want to do <end>',
 '<start> is all right with me <end>',
 '<start> cause you make me feel so brand new <end>',
 "<start> and i want to spend my life with you let me say that since , baby , since we've been together <end>",
 '<start> loving you forever <end>',
 '<start> is what i need <end>',
 '<start> let me , be the one you come running to <end>',
 "<start> i'll never be untrue oh baby <end>",
 "<start> let's , let's stay together gether <end>"]

#### 예제에서는 토큰수 15개 이상일 경우 제외를 권장하였다. 해당 글을 읽고 제외하는 방법이 2가지가 있다고 생각이 들어 하나씩 실험을 해보려 한다.

- 첫 번째는, 토큰화한 후 start와 end를 포함한 길이가 17개가 넘어서는 토큰은 모두 제외한다.

- 두 번째, maxlen을 17로 설정한 뒤 작문과 val_loss값을 확인한다.

- 세 번째, maxlen을 25로 설정한 뒤 작문과 val_loss값을 확인한다.

- 첫번째 방법을 사용하면 전체 데이터셋에서 13000여개가 제외되나, 두번째, 세번째 방법을 사용하면 데이터셋의 수는 동일하게 유지된다.

In [6]:
# <start>와 <end>를 포함하여 17개 이상의 문장은 제외
corpus15 = []
for i in range(len(corpus)):
  if len(corpus[i].split()) <= 17:
    corpus15.append(corpus[i])
print(corpus15)
  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
# 제외된 데이터셋 크기 확인
len(corpus15)

165330

#### pre_sequences 파라미터
sequences: 리스트의 리스트로, 각 성분이 시퀀스이다       
maxlen: 정수, 모든 시퀀스의 최대 길이를 설정하여 제한한다. 10을 넣으면 10보다 큰 것들은 자른다.      
dtype: 출력 시퀀스의 자료형. 가변적 길이의 문자열로 시퀀스를 패딩 하려면object를 사용하시면 됩니다.      
padding: 문자열이 들어간다, 'pre'가 디폴트 값으로 앞쪽에 0이 추가되고, 'post'는 뒤쪽으로 0이 추가되어 각 시퀀스를 패딩 한다.      
truncating: 문자열, 'pre'는 길이가 초과됐을 때 앞쪽을 자르고 'post'는 maxlen보다 큰 시퀀스의 끝의 값들을 제거한다      
value: 부동소수점 혹은 문자열, 패딩 할 값.      

In [8]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다

def tokenize(corpus15):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus15)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus15)   

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='pre')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus15)

[[   0    0    0 ...   28    7    3]
 [   0    0    0 ...   10   44    3]
 [   0    0    0 ...   28   11    3]
 ...
 [   0    0    0 ...   23 3462    3]
 [   0    0    0 ...  593  832    3]
 [   0    0    0 ... 2627  214    3]] <keras_preprocessing.text.Tokenizer object at 0x7f2aed54d550>


In [9]:
# 단어 사전이 어떻게 구축 되어 있는지 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break
#<start>가 2번 이기 때문에 모든 행의 시작은 2였다.

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : the
6 : i
7 : you
8 : and
9 : a
10 : to


# 4. 데이터셋 분리

In [10]:
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]  

In [11]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state=124, shuffle=tgt_input)

In [12]:
print(enc_train.shape)
print(enc_val.shape)
print(dec_train.shape)
print(dec_val.shape)


(132264, 16)
(33066, 16)
(132264, 16)
(33066, 16)


# 5. 인공지능 만들기

In [55]:
BUFFER_SIZE = len(enc_train) # 완벽한 셔플링을 위해서는 전체 데이터 세트의 크기보다 크거나 같은 버퍼 크기가 필요
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)



In [56]:
BUFFER_SIZE = len(enc_val) # 완벽한 셔플링을 위해서는 전체 데이터 세트의 크기보다 크거나 같은 버퍼 크기가 필요
BATCH_SIZE = 256
steps_per_epoch = len(enc_val) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다

dataset_test = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_test = dataset_test.shuffle(BUFFER_SIZE)
dataset_test = dataset_test.batch(BATCH_SIZE, drop_remainder=True)



In [63]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
     
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [60]:
# epoch 마다 결과를 볼 수 있게끔 클래스 생성
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch,logs=None):
      print(generate_text(model3, tokenizer3, init_sentence="<start> i"))

# 6. 인공지능 학습하기

## 6-1 토큰의 길이가 17개를 넘어서는 문장 제외

In [61]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True, dropout=0.3)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True, dropout=0.3)
        self.rnn_3 = tf.keras.layers.LSTM(hidden_size, return_sequences=True, dropout=0.2)
        self.linear2 = tf.keras.layers.Dense(5120)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.rnn_3(out)
        out = self.linear2(out)
        out = self.linear(out)
        return out
    
embedding_size = 128
hidden_size = 512
model2 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [62]:
from sklearn.utils import validation
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=7, validation_data = dataset_test, callbacks=[CustomCallback()])

Epoch 1/7
516/516 [==============================] - 26s 42ms/step - loss: 2.6769 - val_loss: 2.6608
Epoch 2/7
516/516 [==============================] - 20s 39ms/step - loss: 2.6077 - val_loss: 2.6213
Epoch 3/7
516/516 [==============================] - 20s 39ms/step - loss: 2.5465 - val_loss: 2.5897
Epoch 4/7
516/516 [==============================] - 20s 39ms/step - loss: 2.5008 - val_loss: 2.5628
Epoch 5/7
516/516 [==============================] - 20s 39ms/step - loss: 2.4627 - val_loss: 2.5437
Epoch 6/7
516/516 [==============================] - 20s 39ms/step - loss: 2.4217 - val_loss: 2.5239
Epoch 7/7
516/516 [==============================] - 20s 39ms/step - loss: 2.3881 - val_loss: 2.5077


In [64]:
generate_text(model, tokenizer, init_sentence="<start> i")

"<start> i don't know what i want to be , i don't know what you want <end> "

In [65]:
generate_text(model, tokenizer, init_sentence="<start> i love")

"<start> i love you , i don't know what i want to be your favorite girl <end> "

- val_loss값은 2.5077로 2.2 이하로는 내려가지 않았다. 더불어 시작 값을 i로 두었을 때는 같은 문장을 반복하는 모습을 보여준다.
- 시작 값을 i love로 두었을 때는 문맥은 조금 어색하나, 작문실력은 양호한 편인것 같다.

------

## 6-2 maxlen이 17일 때

In [66]:
corpus2 = []

for sentence2 in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence2) == 0: continue
    if sentence2[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence2)
    corpus2.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus2[:10]

["<start> let's stay together i , i'm i'm so in love with you <end>",
 '<start> whatever you want to do <end>',
 '<start> is all right with me <end>',
 '<start> cause you make me feel so brand new <end>',
 "<start> and i want to spend my life with you let me say that since , baby , since we've been together <end>",
 '<start> loving you forever <end>',
 '<start> is what i need <end>',
 '<start> let me , be the one you come running to <end>',
 "<start> i'll never be untrue oh baby <end>",
 "<start> let's , let's stay together gether <end>"]

In [67]:
def tokenize2(corpus2):
    tokenizer2 = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer2.fit_on_texts(corpus2)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor2 = tokenizer2.texts_to_sequences(corpus2)   

    tensor2 = tf.keras.preprocessing.sequence.pad_sequences(tensor2, padding='post', maxlen=17, truncating = 'post')  
    
    print(tensor2,tokenizer2)
    return tensor2, tokenizer2

tensor2, tokenizer2 = tokenize2(corpus2)

[[  2 217 227 ...   0   0   0]
 [  2 615   7 ...   0   0   0]
 [  2  23  22 ...   0   0   0]
 ...
 [  2 208   1 ...   0   0   0]
 [  2  70   7 ...   0   0   0]
 [  2  14   5 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f2a1a7b8450>


In [71]:
# 단어 사전이 어떻게 구축 되어 있는지 확인
for idx in tokenizer2.index_word:
    print(idx, ":", tokenizer2.index_word[idx])

    if idx >= 10: break
#<start>가 2번 이기 때문에 모든 행의 시작은 2였다.

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : the
6 : i
7 : you
8 : and
9 : to
10 : a


In [72]:
# 생성된 텐서를 소스와 타겟으로 분리

# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input2 = tensor2[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input2 = tensor2[:, 1:]    

print(src_input2[0])
print(tgt_input2[0])

[  2 217 227 289   6   4  20  20  28  14  31  29   7   3   0   0]
[217 227 289   6   4  20  20  28  14  31  29   7   3   0   0   0]


In [73]:
enc_train2, enc_val2, dec_train2, dec_val2 = train_test_split(src_input2, tgt_input2, test_size = 0.3, random_state=124, shuffle=tgt_input2)

In [74]:
BUFFER_SIZE2 = len(src_input2) # 완벽한 셔플링을 위해서는 전체 데이터 세트의 크기보다 크거나 같은 버퍼 크기가 필요
BATCH_SIZE2 = 256
steps_per_epoch2 = len(src_input2) // BATCH_SIZE2

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE2 = tokenizer2.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다

dataset2 = tf.data.Dataset.from_tensor_slices((src_input2, tgt_input2))
dataset2 = dataset2.shuffle(BUFFER_SIZE2)
dataset2 = dataset2.batch(BATCH_SIZE2, drop_remainder=True)

In [75]:
BUFFER_SIZE2 = len(enc_val2) # 완벽한 셔플링을 위해서는 전체 데이터 세트의 크기보다 크거나 같은 버퍼 크기가 필요
BATCH_SIZE2 = 256
steps_per_epoch2 = len(enc_val2) // BATCH_SIZE2

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE2 = tokenizer2.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다

dataset_test2 = tf.data.Dataset.from_tensor_slices((enc_val2, dec_val2))
dataset_test2 = dataset_test2.shuffle(BUFFER_SIZE2)
dataset_test2 = dataset_test2.batch(BATCH_SIZE2, drop_remainder=True)

In [76]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True, dropout=0.3)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True, dropout=0.3)
        self.rnn_3 = tf.keras.layers.LSTM(hidden_size, return_sequences=True, dropout=0.2)
        self.linear2 = tf.keras.layers.Dense(5120)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.rnn_3(out)
        out = self.linear2(out)
        out = self.linear(out)
        return out
    
embedding_size = 128
hidden_size = 512
model2 = TextGenerator(tokenizer2.num_words + 1, embedding_size , hidden_size)

In [78]:
from sklearn.utils import validation
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model2.compile(loss=loss, optimizer=optimizer)
model2.fit(dataset2, epochs=7, validation_data = dataset_test2, callbacks=[CustomCallback()])

Epoch 1/7
686/686 [==============================] - 187s 265ms/step - loss: 2.8561 - val_loss: 2.6752
Epoch 2/7
686/686 [==============================] - 181s 263ms/step - loss: 2.6608 - val_loss: 2.4900
Epoch 3/7
686/686 [==============================] - 181s 263ms/step - loss: 2.5116 - val_loss: 2.3335
Epoch 4/7
686/686 [==============================] - 181s 263ms/step - loss: 2.3854 - val_loss: 2.1948
Epoch 5/7
686/686 [==============================] - 181s 263ms/step - loss: 2.2724 - val_loss: 2.0723
Epoch 6/7
686/686 [==============================] - 180s 263ms/step - loss: 2.1689 - val_loss: 1.9605
Epoch 7/7
686/686 [==============================] - 184s 268ms/step - loss: 2.0776 - val_loss: 1.8603


In [79]:
generate_text(model2, tokenizer2, init_sentence="<start> i")

"<start> i don't know what to do <end> "

In [80]:
generate_text(model2, tokenizer2, init_sentence="<start> i love")

'<start> i love you so much <end> '

- maxlen을 17로 두었을 때에는 val_loss값이 1.8603으로 감소하였다.

- 1번 방법과 2번 방법의 나머지 값을 동일하게 놨을 경우 문장을 제외하는 것보다는 maxlen을 사용해 끊어주는 방식이 더 val_loss를 낮출 수 있었다.

- i와 i love를 시작 값으로 놨을 때에도 양호한 작문 실력을 보여주고 있다.

-------

## 6-3 maxlen이 25일 경우

In [100]:
corpus3 = []

for sentence3 in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence3) == 0: continue
    if sentence3[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence3)
    corpus3.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus3[:10]

["<start> let's stay together i , i'm i'm so in love with you <end>",
 '<start> whatever you want to do <end>',
 '<start> is all right with me <end>',
 '<start> cause you make me feel so brand new <end>',
 "<start> and i want to spend my life with you let me say that since , baby , since we've been together <end>",
 '<start> loving you forever <end>',
 '<start> is what i need <end>',
 '<start> let me , be the one you come running to <end>',
 "<start> i'll never be untrue oh baby <end>",
 "<start> let's , let's stay together gether <end>"]

In [101]:
def tokenize3(corpus3):
    tokenizer3 = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer3.fit_on_texts(corpus3)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor3 = tokenizer3.texts_to_sequences(corpus3)   

    tensor3 = tf.keras.preprocessing.sequence.pad_sequences(tensor3, padding='post', maxlen=25, truncating = 'post')  
    
    print(tensor3,tokenizer3)
    return tensor3, tokenizer3

tensor3, tokenizer3 = tokenize3(corpus3)

[[  2 217 227 ...   0   0   0]
 [  2 615   7 ...   0   0   0]
 [  2  23  22 ...   0   0   0]
 ...
 [  2 208   1 ...   0   0   0]
 [  2  70   7 ...   0   0   0]
 [  2  14   5 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f29957c7390>


In [102]:
# 단어 사전이 어떻게 구축 되어 있는지 확인
for idx in tokenizer3.index_word:
    print(idx, ":", tokenizer3.index_word[idx])

    if idx >= 10: break
#<start>가 2번 이기 때문에 모든 행의 시작은 2였다.

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : the
6 : i
7 : you
8 : and
9 : to
10 : a


In [103]:
# 생성된 텐서를 소스와 타겟으로 분리

# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input3 = tensor3[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input3 = tensor3[:, 1:]    

print(src_input3[0])
print(tgt_input3[0])

[  2 217 227 289   6   4  20  20  28  14  31  29   7   3   0   0   0   0
   0   0   0   0   0   0]
[217 227 289   6   4  20  20  28  14  31  29   7   3   0   0   0   0   0
   0   0   0   0   0   0]


In [104]:
enc_train3, enc_val3, dec_train3, dec_val3 = train_test_split(src_input3, tgt_input3, test_size = 0.27, random_state=501, shuffle=tgt_input3)

In [105]:
BUFFER_SIZE3 = len(src_input3) # 완벽한 셔플링을 위해서는 전체 데이터 세트의 크기보다 크거나 같은 버퍼 크기가 필요
BATCH_SIZE3 = 128
steps_per_epoch3 = len(src_input3) // BATCH_SIZE3

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE3 = tokenizer3.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset3 = tf.data.Dataset.from_tensor_slices((src_input3, tgt_input3))
dataset3 = dataset3.shuffle(BUFFER_SIZE3)
dataset3 = dataset3.batch(BATCH_SIZE3, drop_remainder=True)

In [106]:
BUFFER_SIZE3 = len(enc_val3) # 완벽한 셔플링을 위해서는 전체 데이터 세트의 크기보다 크거나 같은 버퍼 크기가 필요
BATCH_SIZE3 = 128
steps_per_epoch3 = len(enc_val3) // BATCH_SIZE3

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE3 = tokenizer3.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다

dataset_test3 = tf.data.Dataset.from_tensor_slices((enc_val3, dec_val3))
dataset_test3 = dataset_test3.shuffle(BUFFER_SIZE3)
dataset_test3 = dataset_test3.batch(BATCH_SIZE3, drop_remainder=True)

In [107]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True, dropout=0.3)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True, dropout=0.3)
        self.rnn_3 = tf.keras.layers.LSTM(hidden_size, return_sequences=True, dropout=0.2)
        self.linear2 = tf.keras.layers.Dense(5120)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.rnn_3(out)
        out = self.linear2(out)
        out = self.linear(out)
        return out
    
embedding_size = 128
hidden_size = 512
model2 = TextGenerator(tokenizer3.num_words + 1, embedding_size , hidden_size)

In [108]:
model3.compile(loss=loss, optimizer=optimizer)
model3.fit(dataset3, epochs=3, validation_data = dataset_test3,callbacks=[CustomCallback()])

Epoch 1/3
1373/1373 [==============================] - 136s 98ms/step - loss: 1.5535 - val_loss: 1.4298
Epoch 2/3
1373/1373 [==============================] - 132s 96ms/step - loss: 1.5418 - val_loss: 1.4140
Epoch 3/3
1373/1373 [==============================] - 133s 96ms/step - loss: 1.5320 - val_loss: 1.4045


In [109]:
generate_text(model3, tokenizer3, init_sentence="<start> i")

'<start> i know that i was a combination of <unk> <end> '

In [110]:
generate_text(model3, tokenizer3, init_sentence="<start> i love")

'<start> i love you liberian girl <end> '

In [111]:
generate_text(model3, tokenizer3, init_sentence="<start> because ")

'<start> because i was a little bit of a <unk> <end> '

- 3번째 모델의 epoch값은 모델 학습 시간 단축과 epoch이 늘어날 수록 작문을 함에 있어 반복적인 문장을 생성하기 때문에 값을 3으로 줄였다.  
- maxlen의 값을 올릴 수록 val_loss가 감소하는 모습을 볼 수 있다.  
- 학습하지 않은 단어를 생성하려고는 하나 작문 실력역시 양호한 편을 보여준다.

----------
# 7. 회고

- 여러번의 시도가 있었지만 최종적으로 가장 좋은 결과를 도출한 것은 Dense layer를 추가한 방식이었다.   
  해당 이유에 대해서 고민해 보자면 Dense 레이어는 추출된 정보를 하나의 일렬로 정렬해 원하는 차원으로 축소하는 개념인데  
  LSTM층이 끝나고 차원을 축소하는 과정에서 한 번에 많은 차원을 축소하다 보니 손실 혹은 학습에 악영향을 미치는 것 같다.  
  좀 더 찾아보니 Dense layer가 하나 만 있다면 조정할 가중치 값들이 너무 사라지다 보니 미세 조정이 어렵다고 한다.  
  딥러닝의 레이어 층에 대해서 조금씩 알가는것이 나름 재밌는 과정인것 같다.  

- 모델을 토큰화 기준 15개로 자르는 첫 번째 과정이 maxlen을 통해 자르는 2번째 모델 보다 더 뛰어난 작문실력이 좋을거라고 예상하였다.   
  왜냐하면 maxlen을 통해서 문장을 자르는 것은 15개가 넘어가는 문장 중간을 자르기 때문에 문맥이 이상하게 학습이 될 것이라고 생각하였기 때문이다.  
  
  그러나 예상과는 다르게 maxlen을 활용하는 것이 작문 실력이 더 좋았으며, loss값 또한 더 낮았다.  
  명확한 이유는 알지 못 하였지만 간략히 생각해보면 데이터셋의 크기 차이가 원인일 수 있다고 생각이 들었다.

- maxlen의 값을 늘리는 것이 loss값이나 작문을 하는 것에 있어서 더 좋은 성능을 보임을 알 수 있다.  
  해당 이유를 생각해보면 이 데이터셋 자체가 잘 형성되어 있어서 전처리를 최소화할 수록 좋은 것 같다.
